# Lab 3

The illusion of conversation, and multi-shot prompting

In [ ]:
from dotenv import load_dotenv
from litellm import completion
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic.*")
load_dotenv(override=True)

In [ ]:
messages

In [2]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi there!"}
]

In [ ]:
response = completion(model="gpt-4.1-mini", messages=messages)
print(response.choices[0].message.content)

In [ ]:
messages = [
    {"role": "user", "content": "My name is Ryan."}
]
response = completion(model="gpt-4.1-mini", messages=messages)
print(response.choices[0].message.content)

In [ ]:
messages = [
    {"role": "user", "content": "What's my name?"}
]
response = completion(model="gpt-4.1-mini", messages=messages)
print(response.choices[0].message.content)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi there!"},
    {"role": "assistant", "content": "Hi there! How can I help you today?"},
    {"role": "user", "content": "My name is Ryan."},
    {"role": "assistant", "content": "Hi Ryan! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
]

response = completion(model="gpt-4.1-mini", messages=messages)
print(response.choices[0].message.content)

## Explanation

The reason I show you this is to emphasize that every call to the LLM is completely stateless.

The sense that we are engaged in a chat with the LLM that takes place over several minutes is an illusion, enabled by the trick of passing in the entire conversation so far.

In [ ]:
# Let's look at the cost - this is the cost of the entire conversation
# Any guesses on how much this is?

print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

## Inference time "training" with multi-shot prompting

In [ ]:
system_prompt = """
You are a helpful assistant that answers questions in English and then in Spanish.
"""
user_prompt = "What's the meaning of life?"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]
response = completion(model="gpt-4.1-mini", messages=messages)
display(Markdown(response.choices[0].message.content))


In [ ]:
system_prompt = """
You are a helpful assistant that answers questions in English and then in Spanish.

For example, if the user asks 'What is 2+2' then you would answer as follows:

## My reply in English is:
Four

## My reply in Spanish is:
Cuatro

"""
user_prompt = "What's the meaning of life?"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]
response = completion(model="gpt-4.1-mini", messages=messages)
display(Markdown(response.choices[0].message.content))

## Is this "training"?

It appears that the model _learned_ the format that we wanted, based on the example given.

But there was no training going on in the Data Science sense; the trillions of parameters in gpt-4.1-mini weren't changed.

It's simply that when predicting the next tokens to come after our question, the model is more likely to predict tokens that would follow the same pattern as the examples in the input.